In [75]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import os, os.path 
import numpy 
import pickle
import tqdm
from tqdm import trange
from glob import glob

"""Change to the data folder"""
new_path = "./new_train/new_train"

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)

In [6]:
pkl_list=glob(os.path.join(new_path, '*'))
# pkl_path = pkl_list[0]
# with open(pkl_path, 'rb') as f:
#     data = pickle.load(f)
#     print(data)

### Create a loader to enable batch processing

In [3]:
batch_sz = 4

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    inp = torch.LongTensor(inp)
    out = torch.LongTensor(out)
    return [inp, out]

val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

In [100]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(60*19*4, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 48)
        self.fc4 = nn.Linear(48, 16)
        self.fc5 = nn.Linear(16, 60*30*4)
  
    def forward(self, x):
        x = torch.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x
    
net = Model()
#net(x) is the same as net.forward(x)

In [103]:

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.5)
print(sum(p.numel() for p in net.parameters()))

418752


### Visualize the batch of sequences

In [ ]:
# Train

device = "cuda"
net = Model().to(device)
print(sum(p.numel() for p in net.parameters()))
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    net.train()
    for i, data in enumerate(val_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data #input has shape BxCxHxW and labels has shape B, print(inputs.shape) is a very useful function
        

        inputs = torch.tensor(inputs, dtype=torch.float)
        labels = torch.tensor(labels, dtype=torch.float)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        for mset, mlabel in zip(inputs, labels):
            outputs = net(mset.cuda()) #net.forward(inputs)
            outputs = torch.reshape(outputs, (60, 30, 4))
            loss = torch.mean((outputs-mlabel.cuda())**2)  # loss = torch.mean((ouputs - labels)**2)+74
            loss.backward()
            optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

418752


<ipython-input-3-e34a6e23e746>:7: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  inp = torch.LongTensor(inp)
<ipython-input-3-e34a6e23e746>:8: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  out = torch.LongTensor(out)
<ipython-input-104-3612a50b1d4d>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs, dtype=torch.float)
<ipython-input-104-3612a50b1d4d>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTe

[1,   100] loss: 17878.972
[1,   200] loss: 17560.708
[1,   300] loss: 19011.515
[1,   400] loss: 14556.279
[1,   500] loss: 16279.071
[1,   600] loss: 14411.552
[1,   700] loss: 14721.864
[1,   800] loss: 16019.952
[1,   900] loss: 20141.909
[1,  1000] loss: 16329.548
[1,  1100] loss: 17104.844
[1,  1200] loss: 17198.701
[1,  1300] loss: 18059.247
[1,  1400] loss: 16198.881
[1,  1500] loss: 16178.670
[1,  1600] loss: 15650.260


In [117]:
# I think inp is p_x, p_y, v_x, v_y for 60 objects over 19 timestamps
# inp = torch.reshape(inp, (4,60*19*4))
inp = torch.reshape(inp, (2,60*19*4))
inp = torch.tensor(inp, dtype=torch.float)
# Out is the same but for 30r timestamps
predict = net.predict(inp)
predict = torch.reshape(predict, (2,60,30,4))
print(predict-out)

Predicted data based on trained weights: 
tensor([[[[-2.8910e+03, -1.3570e+03, -8.0000e+00, -6.0000e+00],
          [-2.8920e+03, -1.3580e+03, -7.0000e+00, -7.0000e+00],
          [-2.8920e+03, -1.3590e+03, -6.0000e+00, -7.0000e+00],
          ...,
          [-2.9130e+03, -1.3770e+03, -7.0000e+00, -7.0000e+00],
          [-2.9130e+03, -1.3780e+03, -7.0000e+00, -7.0000e+00],
          [-2.9140e+03, -1.3790e+03, -7.0000e+00, -6.0000e+00]],

         [[-2.8240e+03, -1.3040e+03,  7.0000e+00,  5.0000e+00],
          [-2.8240e+03, -1.3040e+03,  6.0000e+00,  6.0000e+00],
          [-2.8230e+03, -1.3030e+03,  5.0000e+00,  5.0000e+00],
          ...,
          [-2.8040e+03, -1.2860e+03,  9.0000e+00,  6.0000e+00],
          [-2.8030e+03, -1.2850e+03,  7.0000e+00,  8.0000e+00],
          [-2.8030e+03, -1.2840e+03,  6.0000e+00,  9.0000e+00]],

         [[-2.8980e+03, -1.3600e+03, -1.0000e+01, -8.0000e+00],
          [-2.8990e+03, -1.3610e+03, -1.0000e+01, -9.0000e+00],
          [-2.9000e+03, -1.3

<ipython-input-117-630361ec44c3>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inp = torch.tensor(inp, dtype=torch.float)
